In [1]:
# -*- coding: utf-8
# Abraji (https://www.abraji.org.br)
# Reinaldo Chaves (reinaldo@abraji.org.br)
# Programa 04 do curso Como usar Python para descobrir uma rede de interesses
#

In [2]:
import pandas as pd
import numpy as np
import unidecode

# Verifica se os sócios de indústrias farmacêuticas foram doadoras de campanha em 2010, 2014 e 2018

# Aponta quais beneficiários foram eleitos em 2018

Formato de valores com decimais no float

In [3]:
pd.options.display.float_format = '{:,.2f}'.format

Função para retirar acentos - útil para comparar nomes

In [4]:
def f(str):
    return (unidecode.unidecode(str))

# Carrega sócios

In [6]:
farma_socios = pd.read_csv('resultados/empresas_farm_com_socios.csv',\
                           sep=',',\
                           encoding = 'utf_8',\
                           converters={'cnpj': lambda x: str(x),
                                       'cnae_fiscal': lambda x: str(x),
                                       'cpf_representante_legal': lambda x: str(x),
                                       'nome_cidade_exterior': lambda x: str(x),
                                       'nm_pais': lambda x: str(x)
                                      },\
                           decimal=',')

In [7]:
farma_socios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201085 entries, 0 to 201084
Data columns (total 20 columns):
cnpj                           201085 non-null object
identificador_matrizfilial     201085 non-null int64
razao_social                   201085 non-null object
nome_fantasia                  141779 non-null object
situacao_cadastral             201085 non-null int64
data_situacao_cadastral        201085 non-null object
nome_cidade_exterior           201085 non-null object
codigo_natureza_juridica       201085 non-null int64
data_inicio_atividade          201085 non-null object
cnae_fiscal                    201085 non-null object
uf                             201085 non-null object
municipio                      201085 non-null object
qualificacao_do_responsavel    201085 non-null int64
capital_social                 201085 non-null int64
nome_socio                     201085 non-null object
cnpj_cpf_do_socio              201085 non-null object
codigo_qualificacao_socio     

In [8]:
farma_socios.head()

,cnpj,identificador_matrizfilial,razao_social,nome_fantasia,situacao_cadastral,data_situacao_cadastral,nome_cidade_exterior,codigo_natureza_juridica,data_inicio_atividade,cnae_fiscal,uf,municipio,qualificacao_do_responsavel,capital_social,nome_socio,cnpj_cpf_do_socio,codigo_qualificacao_socio,percentual_capital_social,cpf_representante_legal,nome_representante_legal
0,00002903000101,1,DROGARIA SANTA RITA DE CANDIDO MOTA LTDA,NaN,2,2004-01-23,,2062,1994-05-12,4771701,SP,CANDIDO MOTA,49,200000,SYLVIO RENATO BEGOSSO,***830948**,49,0,,NaN
1,00002903000101,1,DROGARIA SANTA RITA DE CANDIDO MOTA LTDA,NaN,2,2004-01-23,,2062,1994-05-12,4771701,SP,CANDIDO MOTA,49,200000,LIGIA RORATO BEGOSSO PESCADA,***518748**,49,0,,NaN
2,00003230000104,1,FOLTRAN & NATALIO LTDA,FARMACIA BELA VISTA,2,2000-12-01,,2062,1994-05-12,4771701,SP,PIRACICABA,49,0,WILSON ANTONIO FOLTRAM,***224398**,49,0,,NaN
3,00003230000104,1,FOLTRAN & NATALIO LTDA,FARMACIA BELA VISTA,2,2000-12-01,,2062,1994-05-12,4771701,SP,PIRACICABA,49,0,LUIS FERNANDO NATALIO,***446358**,49,0,,NaN
4,00005391000137,1,EZIO PIRES EIRELI,NaN,2,2005-11-03,,2305,1994-05-17,4771701,SP,FRANCA,65,10689400,EZIO PIRES,***966118**,65,0,,NaN


O CPF tem sempre 11 dígitos

O formato do campo cnpjcpf_do_socio é: 3 asteriscos + os dígitos 456789 do CPF + 2 asteriscos

# Acessa resultado de eleicoes 2018

In [9]:
candidatos = pd.read_csv('resultados/candidatos_2018.csv',\
                         sep=',',\
                         encoding = 'utf-8',\
                         converters={'SQ_CANDIDATO': lambda x: str(x),
                                     'NR_CPF_CANDIDATO': lambda x: str(x),
                                     'NR_TITULO_ELEITORAL_CANDIDATO': lambda x: str(x),
                                    })

identificador = candidatos['DS_SIT_TOT_TURNO'].isin(['ELEITO', 
                                                     'ELEITO POR MÉDIA',
                                                     'ELEITO POR QP']) 

eleitos = candidatos[identificador]

# Carrega doacoes

2010

In [10]:
doacoes_2010 = pd.read_csv('resultados/doacoes_2010.csv',\
                           sep=',',\
                           encoding = 'utf_8',\
                           converters={'cpf_candidato': lambda x: str(x),
                                       'sequencial': lambda x: str(x),
                                       'cpf_cnpj_doador': lambda x: str(x),
                                       'nome_doador': lambda x: str(x)},
                           decimal=',')

In [11]:
def limpeza(linha):
    documento = str(linha["cpf_cnpj_doador"])
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf"

doacoes_2010['cnpj_raiz'] = doacoes_2010.apply(limpeza, axis=1)

In [12]:
# Separa dataframe apenas com doações vindas de CPFs

In [13]:
doacoes_2010_pf = doacoes_2010[(doacoes_2010['cnpj_raiz'] == 'documento_cpf')].copy()

In [14]:
# Cria nova coluna com o CPF mascarado dos doadores
# 11 dígitos
# 12345678901
# 01234567890 -> python começa contar pelo zero
# ***345678**

# List slicing
# [start:end]
# inclusive:exclusive

In [16]:
doacoes_2010_pf['cpf_mascarado']='***'+doacoes_2010_pf['cpf_cnpj_doador'].str[3:9]+'**'

In [17]:
# Retira acentos dos nomes

In [19]:
doacoes_2010_pf["nome_doador"] = doacoes_2010_pf["nome_doador"].apply(f)

In [38]:
doacoes_2010_pf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 279895 entries, 9 to 427011
Data columns (total 21 columns):
data_hora            279895 non-null object
sequencial           279895 non-null object
uf                   279895 non-null object
partido_eleicao      279895 non-null object
numero_candidato     279895 non-null int64
cargo                279895 non-null object
nome_candidato       279895 non-null object
cpf_candidato        279895 non-null object
entrega_conjunto     279895 non-null object
recibo_eleitoral     279139 non-null float64
numero_documento     117579 non-null object
cpf_cnpj_doador      279895 non-null object
nome_doador          279895 non-null object
data_receita         279895 non-null object
valor_receita        279895 non-null float64
tipo_receita         279895 non-null object
fonte_recurso        279895 non-null object
especie_recurso      279895 non-null object
descricao_receita    279895 non-null object
cnpj_raiz            279895 non-null object
cpf_masc

In [39]:
doacoes_2010_pf.reset_index().tail()

,index,data_hora,sequencial,uf,partido_eleicao,numero_candidato,cargo,nome_candidato,cpf_candidato,entrega_conjunto,...,cpf_cnpj_doador,nome_doador,data_receita,valor_receita,tipo_receita,fonte_recurso,especie_recurso,descricao_receita,cnpj_raiz,cpf_mascarado
279890,427002,13/11/201701:02:20,110000000577,MT,PDT,12345,Deputado Estadual,JOSÉ ANTONIO GONÇALVES VIANA,29805600904,Nao,...,46612343168,DANIEL FAUSTINO DA COSTA,02/10/2010,500.00,Recursos de pessoas físicas,Nao especificado,Estimado,"CESSAO DE 1 MOTOCICLETA P/ CAMPANHA ELEITORAL,...",documento_cpf,***123431**
279891,427003,13/11/201701:02:20,110000000577,MT,PDT,12345,Deputado Estadual,JOSÉ ANTONIO GONÇALVES VIANA,29805600904,Nao,...,20743106172,CARLOS ALBERTO HORTENCI DE BARROS,02/10/2010,"2,343.00",Recursos de pessoas físicas,Nao especificado,Estimado,"CESSAO DE 1 VEICULO P/ CAMPANHA ELEITORAL, R$2...",documento_cpf,***431061**
279892,427004,13/11/201701:02:20,110000000577,MT,PDT,12345,Deputado Estadual,JOSÉ ANTONIO GONÇALVES VIANA,29805600904,Nao,...,62137425149,MARCO ANTONIO DOS SANTOS AMARAL,02/10/2010,"1,761.00",Recursos de pessoas físicas,Nao especificado,Estimado,"CESSAO DE 1 VEICULO P/ CAMPANHA ELEITORAL, R$ ...",documento_cpf,***374251**
279893,427010,13/11/201701:02:20,110000000578,MT,PPS,23555,Deputado Estadual,ANTONIO SANTANA CORREA MARQUES JUNIOR,95776060168,Nao,...,52305660197,MARCO LEVY RODRIGUES DO PRADO,06/09/2010,600.00,Recursos de pessoas físicas,Nao especificado,Estimado,PRODUÇÃO DE JINGLE P/CAMPANHA EM POCONE-MT ...,documento_cpf,***056601**
279894,427011,13/11/201701:02:20,110000000578,MT,PPS,23555,Deputado Estadual,ANTONIO SANTANA CORREA MARQUES JUNIOR,95776060168,Nao,...,55132960987,OSVALDO AGUIAR DE AZEVEDO,22/09/2010,"1,200.00",Recursos de pessoas físicas,Nao especificado,Estimado,"30 ADESIVOS PERFURADOS P/VIDRO, 5 CONJUNTOS DE...",documento_cpf,***329609**


Doações diretas de CPF

In [40]:
# Compara ao mesmo tempo nome do doador e o CPF mascarado

In [20]:
farma_doadores_diretos_cpf_2010 = pd.merge(doacoes_2010_pf, 
                                           farma_socios.drop_duplicates('nome_socio'), 
                                           left_on=['nome_doador', 'cpf_mascarado'], 
                                           right_on=['nome_socio', 'cnpj_cpf_do_socio'])

In [21]:
print('O dataframe contém ' + str(farma_doadores_diretos_cpf_2010.shape[0]) + ' linhas')

O dataframe contém 562 linhas


In [22]:
# Adiciona informações se o candidato foi eleito em 2018

In [23]:
farma_doadores_diretos_cpf_2010 = pd.merge(farma_doadores_diretos_cpf_2010, \
                                       eleitos.drop_duplicates('NR_CPF_CANDIDATO'), \
                                       left_on='cpf_candidato', \
                                       right_on='NR_CPF_CANDIDATO',\
                                       how='left')

In [24]:
print('O dataframe contém ' + str(farma_doadores_diretos_cpf_2010.shape[0]) + ' linhas')

O dataframe contém 562 linhas


In [25]:
farma_doadores_diretos_cpf_2010.to_excel('resultados/compara/doacoes_diretas_socios_empresas_2010.xlsx',sheet_name='Sheet1')

Maiores doadores

In [26]:
farma_doadores_diretos_cpf_2010.groupby("nome_doador").valor_receita.sum().reset_index().sort_values("valor_receita", ascending=False).head(10)

,nome_doador,valor_receita
242,MANOEL SALVIANO SOBRINHO,"608,000.00"
245,MARCELO HENRIQUE LIMIRIO GONCALVES,"200,000.00"
193,JOSE LUIZ SANTOS,"117,000.00"
238,LUIZ VILELA PARANAIBA,"102,900.00"
48,AUGUSTO CESAR ELIHIMAS DE CARVALHO,"91,500.00"
171,JOAO SAMPAIO DE OLIVEIRA,"75,838.00"
269,MARIZETE LISBOA FERNANDES PEREIRA,"56,700.00"
25,ANGELO MARIO CORONEL DE AZEVEDO MARTINS,"52,532.42"
49,AUGUSTO FRANCO DINIZ,"51,000.00"
121,FRANCISCO NORBERTO NUNES BARRETO,"50,000.00"


Candidatos que mais receberam

In [27]:
farma_doadores_diretos_cpf_2010.groupby("nome_candidato").valor_receita.sum().reset_index().sort_values("valor_receita", ascending=False).head(10)

,nome_candidato,valor_receita
231,MANOEL SALVIANO SOBRINHO,"608,000.00"
75,DEMOSTENES LAZARO XAVIER TORRES,"200,000.00"
59,CLAUDIA SILVA SANTOS OLIVEIRA,"117,000.00"
223,LUIZ VILELA PARANAIBA,"102,900.00"
43,AUGUSTO CESAR ELIHIMAS DE CARVALHO,"91,500.00"
183,JOÃO SAMPAIO DE OLIVEIRA,"76,788.00"
250,MARIZETE LISBOA FERNANDES PEREIRA,"57,700.00"
24,ANGELO MARIO CORONEL DE AZEVEDO MARTINS,"52,532.42"
291,RODRIGO SOBRAL ROLLEMBERG,"51,000.00"
188,JUSCELINO CARDOSO DE SÁ,"50,000.00"


Tabela dinâmica doadores candidatos empresas

In [28]:
tabela = pd.pivot_table(farma_doadores_diretos_cpf_2010,index=["nome_doador","nome_candidato", "razao_social"], values=["valor_receita"], aggfunc=np.sum)
result = tabela.sort_values('valor_receita', ascending=False).head(10)
result

,,,valor_receita
nome_doador,nome_candidato,razao_social,
MANOEL SALVIANO SOBRINHO,MANOEL SALVIANO SOBRINHO,RELSEFARMA INDUSTRIA FARMACEUTICA LTDA,"608,000.00"
MARCELO HENRIQUE LIMIRIO GONCALVES,DEMOSTENES LAZARO XAVIER TORRES,"NEO MARCAS INDUSTRIA FARMACEUTICA E ALIMENTICIA, COMERCIO E PARTICIPACOES S. A.","200,000.00"
JOSE LUIZ SANTOS,CLAUDIA SILVA SANTOS OLIVEIRA,FARMAFORTE DROGARIA E FARMACIA LTDA,"117,000.00"
LUIZ VILELA PARANAIBA,LUIZ VILELA PARANAIBA,DROGARIA UNILAR LTDA,"102,900.00"
AUGUSTO CESAR ELIHIMAS DE CARVALHO,AUGUSTO CESAR ELIHIMAS DE CARVALHO,EMPRESA BRASILEIRA DE HEMODERIVADOS E BIOTECNOLOGIA - HEMOBRAS,"91,500.00"
JOAO SAMPAIO DE OLIVEIRA,JOÃO SAMPAIO DE OLIVEIRA,PROPEC PRODUTOS PARA PECUARIA LTDA,"75,838.00"
MARIZETE LISBOA FERNANDES PEREIRA,MARIZETE LISBOA FERNANDES PEREIRA,FARMACIA SANTO ANTONIO LTDA,"56,700.00"
ANGELO MARIO CORONEL DE AZEVEDO MARTINS,ANGELO MARIO CORONEL DE AZEVEDO MARTINS,JULY HAIR PRODUTOS DE BELEZA LTDA,"52,532.42"
AUGUSTO FRANCO DINIZ,RODRIGO SOBRAL ROLLEMBERG,CAPITAL MEDH IMPORTACAO DISTRIBUICAO E REPRESENTACAO COMERCIAL LTDA,"51,000.00"


Exemplo de seleção das empresas

In [29]:
nome = farma_doadores_diretos_cpf_2010[(farma_doadores_diretos_cpf_2010['nome_socio'] == 'MANOEL SALVIANO SOBRINHO')]
nome[["razao_social", "cnpj"]]

,razao_social,cnpj
473,RELSEFARMA INDUSTRIA FARMACEUTICA LTDA,04006744000174
474,RELSEFARMA INDUSTRIA FARMACEUTICA LTDA,04006744000174
475,RELSEFARMA INDUSTRIA FARMACEUTICA LTDA,04006744000174
476,RELSEFARMA INDUSTRIA FARMACEUTICA LTDA,04006744000174
477,RELSEFARMA INDUSTRIA FARMACEUTICA LTDA,04006744000174
478,RELSEFARMA INDUSTRIA FARMACEUTICA LTDA,04006744000174
479,RELSEFARMA INDUSTRIA FARMACEUTICA LTDA,04006744000174
480,RELSEFARMA INDUSTRIA FARMACEUTICA LTDA,04006744000174
481,RELSEFARMA INDUSTRIA FARMACEUTICA LTDA,04006744000174
482,RELSEFARMA INDUSTRIA FARMACEUTICA LTDA,04006744000174


Etapas para mostrar os sócios que têm mais de uma empresa

In [30]:
# Separa dataframe apenas com nomes e CPF, sem repetir - keep first

In [31]:
selecao1 = farma_doadores_diretos_cpf_2010.drop_duplicates(['nome_socio'], keep='first')

In [32]:
selecao2 = selecao1[['nome_socio', 'cnpj_cpf_do_socio']].copy()

In [33]:
selecao2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 391 entries, 0 to 561
Data columns (total 2 columns):
nome_socio           391 non-null object
cnpj_cpf_do_socio    391 non-null object
dtypes: object(2)
memory usage: 9.2+ KB


In [34]:
# Cruza esse dataframe com a base original que tem todos os dados

In [35]:
selecao3 = pd.merge(farma_socios, selecao2, left_on=['nome_socio', 'cnpj_cpf_do_socio'], right_on=['nome_socio', 'cnpj_cpf_do_socio'])

In [36]:
selecao3.shape

(1320, 20)

In [37]:
# Seleciona apenas colunas de interesse

In [38]:
selecao4 = selecao3[['cnpj', 'razao_social', 'nome_socio', 'cnpj_cpf_do_socio']].copy()

In [39]:
# Conta as aparições

In [40]:
selecao4['nome_socio'].value_counts()

ALEXANDRE DELLA COLETTA                251
MODESTO CARVALHO DE ARAUJO NETO        221
RICARDO MAEOKA                          73
CARLOS AUGUSTO ROSA                     63
ARNALDO ANGELONI                        23
ANTENOR ANGELONI                        23
ANTONIO JOSE MONTE                      21
GENILSON BRITO DE OLIVEIRA              21
GERMANO JACOME PATRIOTA                 21
ELIAS DANTAS SOUTO                      14
EMANOEL NAZARENO SOUZA MUNIZ            10
FLAVIO GIORGI MEDEIROS OLIVEIRA          9
RUBENS CARLOS DE OLIVEIRA JUNIOR         9
ANSELMO BENNATI SOBRINHO                 8
JAMES SANTOS SOARES                      8
JOSEANE DA SILVA SANTOS                  7
JOSE EVENILDE BENEVIDES MARTINS          6
CARLOS ROGERIO MARINHO DIAS              6
JOSE LUIZ SANTOS                         5
ROGERIO TOKARSKI                         5
VALDETE VIODRES TANAKA                   5
ANTONIO BAREA                            5
ALEXANDRE JOSE DIAS                      5
RAIMUNDO CA

In [41]:
# Exemplo das empresas de um nome

In [42]:
nome = selecao4[(selecao4['nome_socio'] == 'ALEXANDRE DELLA COLETTA')]
nome

,cnpj,razao_social,nome_socio,cnpj_cpf_do_socio
132,02235861000184,DROGARIA DELMAR LTDA,ALEXANDRE DELLA COLETTA,***840798**
133,02235861000265,DROGARIA DELMAR LTDA,ALEXANDRE DELLA COLETTA,***840798**
134,02235861000346,DROGARIA DELMAR LTDA,ALEXANDRE DELLA COLETTA,***840798**
135,02235861000427,DROGARIA DELMAR LTDA,ALEXANDRE DELLA COLETTA,***840798**
136,02476085000104,DROGARIA NOVA CAIEIRAS LTDA,ALEXANDRE DELLA COLETTA,***840798**
137,02743218000161,DROGA EX LTDA,ALEXANDRE DELLA COLETTA,***840798**
138,02743218000242,DROGA EX LTDA,ALEXANDRE DELLA COLETTA,***840798**
139,02743218000323,DROGA EX LTDA,ALEXANDRE DELLA COLETTA,***840798**
140,02743218000404,DROGA EX LTDA,ALEXANDRE DELLA COLETTA,***840798**
141,02743218000595,DROGA EX LTDA,ALEXANDRE DELLA COLETTA,***840798**


In [43]:
selecao4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1320 entries, 0 to 1319
Data columns (total 4 columns):
cnpj                 1320 non-null object
razao_social         1320 non-null object
nome_socio           1320 non-null object
cnpj_cpf_do_socio    1320 non-null object
dtypes: object(4)
memory usage: 51.6+ KB


In [44]:
selecao4.to_excel('resultados/compara/empresas_e_socios_das_empresas_de_2010.xlsx',sheet_name='Sheet1')

2014

In [45]:
doacoes_2014 = pd.read_csv('resultados/doacoes_2014.csv',\
                           sep=',',\
                           encoding = 'utf_8',\
                           converters={'sequencial': lambda x: str(x),
                                       'cpf_candidato': lambda x: str(x),
                                       'cpf_cnpj_doador': lambda x: str(x),
                                       'cpf_cnpj_doador_originario': lambda x: str(x)},\
                           decimal=',')

def limpeza1(linha):
    documento = str(linha["cpf_cnpj_doador"])
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf"

def limpeza2(linha):
    documento = str(linha["cpf_cnpj_doador_originario"])
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf"

doacoes_2014['cnpj_raiz'] = doacoes_2014.apply(limpeza1, axis=1)
doacoes_2014['cnpj_raiz_originario'] = doacoes_2014.apply(limpeza2, axis=1)

In [46]:
doacoes_2014.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 427489 entries, 0 to 427488
Data columns (total 34 columns):
cod_eleicao                          427489 non-null int64
desc_eleicao                         427489 non-null object
data_hora                            427489 non-null object
cnpj_prestador_conta                 427489 non-null int64
sequencial                           427489 non-null object
uf                                   427489 non-null object
partido_eleicao                      427489 non-null object
numero_candidato                     427489 non-null int64
cargo                                427489 non-null object
nome_candidato                       427489 non-null object
cpf_candidato                        427489 non-null object
recibo_eleitoral                     427489 non-null object
numero_documento                     427472 non-null object
cpf_cnpj_doador                      427489 non-null object
nome_doador                          427489 non-null

In [47]:
# Retira acentos

In [48]:
doacoes_2014["nome_doador_receita"] = doacoes_2014["nome_doador_receita"].apply(f)
doacoes_2014["nome_doador_originario_receita"] = doacoes_2014["nome_doador_originario_receita"].apply(f)

In [49]:
# Separa apenas doacoes de PF

In [50]:
doacoes_2014_pf_direta = doacoes_2014[(doacoes_2014['cnpj_raiz'] == 'documento_cpf') & (doacoes_2014['nome_doador_originario'] == '#NULO')].copy()
doacoes_2014_pf_direta.shape

(220464, 34)

In [51]:
# Cria campo de CPF mascarado

In [52]:
doacoes_2014_pf_direta['cpf_cnpj_doador_direto_mascara']='***'+doacoes_2014_pf_direta['cpf_cnpj_doador'].str[3:9]+'**'

In [53]:
# Separa apenas doacoes de PF - originárias

In [54]:
doacoes_2014_pf_indireta = doacoes_2014[(doacoes_2014['cnpj_raiz_originario'] == 'documento_cpf') & (doacoes_2014['nome_doador_originario'] != '#NULO')].copy()
doacoes_2014_pf_indireta.shape

(16793, 34)

In [55]:
# Cria campo de CPF mascarado - doacao originária

In [56]:
doacoes_2014_pf_indireta['cpf_cnpj_doador_indireto_mascara']='***'+doacoes_2014_pf_direta['cpf_cnpj_doador_originario'].str[3:9]+'**'

Doações diretas de CPF

In [57]:
# Compara ao mesmo tempo nome do doador e o CPF mascarado

In [58]:
farma_doadores_diretos_cpf_2014 = pd.merge(doacoes_2014_pf_direta, farma_socios.drop_duplicates('nome_socio'), left_on=['nome_doador_receita', 'cpf_cnpj_doador_direto_mascara'], right_on=['nome_socio', 'cnpj_cpf_do_socio'])

In [59]:
print('O dataframe contém ' + str(farma_doadores_diretos_cpf_2014.shape[0]) + ' linhas')

O dataframe contém 589 linhas


In [60]:
farma_doadores_diretos_cpf_2014 = pd.merge(farma_doadores_diretos_cpf_2014, \
                                       eleitos.drop_duplicates('NR_CPF_CANDIDATO'), \
                                       left_on='cpf_candidato', \
                                       right_on='NR_CPF_CANDIDATO',\
                                       how='left')

In [61]:
print('O dataframe contém ' + str(farma_doadores_diretos_cpf_2014.shape[0]) + ' linhas')

O dataframe contém 589 linhas


In [62]:
farma_doadores_diretos_cpf_2014.to_excel('resultados/compara/doacoes_diretas_socios_empresas_2014.xlsx',sheet_name='Sheet1')

Maiores doadores 

In [63]:
farma_doadores_diretos_cpf_2014.groupby("nome_doador").valor_receita.sum().reset_index().sort_values("valor_receita", ascending=False).head(10)

,nome_doador,valor_receita
222,MARCELO HENRIQUE LIMIRIO GONÇALVES,"1,175,000.00"
276,RAIMUNDO NONATO GOMES RODRIGUES,"200,000.00"
23,ANDERSON BRAGA MIRANDA,"196,000.00"
45,ARMANDEO DE QUEIROZ MONTEIRO FILHO,"176,000.00"
229,MARCIO PASCHOAL GIUDICIO,"175,688.07"
206,LUANA B LIMIRIO GONÇALVES DE SANT ANNA BRAGA,"161,000.00"
208,LUIS CARLOS PINHEIRO FILHO,"142,000.00"
177,JOSE EUSTAQUIO DE FREITAS,"135,000.00"
219,MANOEL SALVIANO SOBRINHO,"123,000.00"
14,ALEXANDRE VICENTE DE PAULA ALMEIDA,"121,000.00"


Candidatos que mais receberam

In [64]:
farma_doadores_diretos_cpf_2014.groupby("nome_candidato").valor_receita.sum().reset_index().sort_values("valor_receita", ascending=False).head(10)

,nome_candidato,valor_receita
11,ALEXANDRE BALDY DE SANT'ANNA BRAGA,"1,336,000.00"
223,LUANA MARIA DA SILVA COSTA,"200,000.00"
22,ANDERSON BRAGA MIRANDA,"196,000.00"
305,RENZO DE VASCONCELOS,"180,000.00"
46,ARMANDO DE QUEIROZ MONTEIRO NETO,"176,000.00"
251,MARCIO PASCHOAL GIUDICIO,"175,688.07"
174,JOSE EUSTAQUIO DE FREITAS,"155,000.00"
226,LUIS CARLOS PINHEIRO FILHO,"142,000.00"
127,FRANCISCO TALES MACEDO JUNIOR,"123,000.00"
12,ALEXANDRE VICENTE DE PAULA ALMEIDA,"121,000.00"


Tabela dinâmica doadores candidatos empresas

In [65]:
tabela = pd.pivot_table(farma_doadores_diretos_cpf_2014,index=["nome_doador","nome_candidato", "razao_social"], values=["valor_receita"], aggfunc=np.sum)
result = tabela.sort_values('valor_receita', ascending=False).head(10)
result

,,,valor_receita
nome_doador,nome_candidato,razao_social,
MARCELO HENRIQUE LIMIRIO GONÇALVES,ALEXANDRE BALDY DE SANT'ANNA BRAGA,"NEO MARCAS INDUSTRIA FARMACEUTICA E ALIMENTICIA, COMERCIO E PARTICIPACOES S. A.","1,175,000.00"
RAIMUNDO NONATO GOMES RODRIGUES,LUANA MARIA DA SILVA COSTA,R N GOMES RODRIGUES & CIA LTDA,"200,000.00"
ANDERSON BRAGA MIRANDA,ANDERSON BRAGA MIRANDA,FARMACIA BRAGA E LIMA 2008 EIRELI,"196,000.00"
ARMANDEO DE QUEIROZ MONTEIRO FILHO,ARMANDO DE QUEIROZ MONTEIRO NETO,ARH EMPREENDIMENTOS E PARTICIPACOES LTDA,"176,000.00"
MARCIO PASCHOAL GIUDICIO,MARCIO PASCHOAL GIUDICIO,DROGARIA GIUDICIO LTDA,"175,688.07"
LUANA B LIMIRIO GONÇALVES DE SANT ANNA BRAGA,ALEXANDRE BALDY DE SANT'ANNA BRAGA,"NEO MARCAS INDUSTRIA FARMACEUTICA E ALIMENTICIA, COMERCIO E PARTICIPACOES S. A.","161,000.00"
LUIS CARLOS PINHEIRO FILHO,LUIS CARLOS PINHEIRO FILHO,DROGARIA E FARMACIA PINHEIRO LTDA,"142,000.00"
JOSE EUSTAQUIO DE FREITAS,JOSE EUSTAQUIO DE FREITAS,NATURE FARMA LTDA,"135,000.00"
MANOEL SALVIANO SOBRINHO,FRANCISCO TALES MACEDO JUNIOR,RELSEFARMA INDUSTRIA FARMACEUTICA LTDA,"123,000.00"


In [66]:
teste = farma_doadores_diretos_cpf_2014[(farma_doadores_diretos_cpf_2014['nome_doador_receita'] == 'MARCELO HENRIQUE LIMIRIO GONCALVES')]

In [67]:
teste[['nome_doador_receita', 'cpf_cnpj_doador_direto_mascara', 'nome_socio', 'cnpj_cpf_do_socio']]

,nome_doador_receita,cpf_cnpj_doador_direto_mascara,nome_socio,cnpj_cpf_do_socio
30,MARCELO HENRIQUE LIMIRIO GONCALVES,***009701**,MARCELO HENRIQUE LIMIRIO GONCALVES,***009701**


In [68]:
selecao1 = farma_doadores_diretos_cpf_2014.drop_duplicates(['nome_socio'], keep='first')
selecao2 = selecao1[['nome_socio', 'cnpj_cpf_do_socio']].copy()
selecao3 = pd.merge(farma_socios, selecao2, left_on=['nome_socio', 'cnpj_cpf_do_socio'], right_on=['nome_socio', 'cnpj_cpf_do_socio'])
selecao4 = selecao3[['cnpj', 'razao_social', 'nome_socio', 'cnpj_cpf_do_socio']].copy()
selecao4['nome_socio'].value_counts()

PEDRO HENRIQUE KAPPAUN BRAIR                 732
LYNTTON JOSE PAIXAO GUEDES                    41
SEBASTIAO RAMILO BULCAO BRINGEL               26
GENILSON BRITO DE OLIVEIRA                    21
RUBIN WENDLAND                                21
JOSE VEIGA VEIGA                              21
ALVARO SILVEIRA JUNIOR                        20
FLAVIO JUNIOR BORGES                          17
CHARLES VIEIRA DA COSTA                       12
GERALDO SOUZA ANDRADE                         11
JOSE DE CASTRO PEREIRA                        10
JOSE DE VASCONCELOS ARAGAO FILHO               9
JOSE RENATO STRAPASSON                         8
CRISTIANO JOSE AZEVEDO CALDEIRA BRANT          6
ALLAN FELIPE FENELON                           6
JOSE LUIZ SANTOS                               5
SEBASTIAO FERRO DE MORAES                      5
AUGUSTO CESAR ELIHIMAS DE CARVALHO             5
CARLOS ALBERTO VIEIRA DA COSTA                 5
ROGERIO TOKARSKI                               5
ANTONIO BAREA       

In [69]:
selecao4.to_excel('resultados/compara/empresas_e_socios_das_empresas_de_2014.xlsx',sheet_name='Sheet1')

Doações indiretas de CPF

In [70]:
farma_doadores_indiretos_cpf_2014 = pd.merge(doacoes_2014_pf_indireta, farma_socios.drop_duplicates('nome_socio'), left_on=['nome_doador_originario_receita', 'cpf_cnpj_doador_indireto_mascara'], right_on=['nome_socio', 'cnpj_cpf_do_socio'])

In [71]:
print('O dataframe contém ' + str(farma_doadores_indiretos_cpf_2014.shape[0]) + ' linhas')

O dataframe contém 0 linhas


2018

In [72]:
# E os mesmos passos para 2018

In [73]:
doacoes_2018 = pd.read_csv('resultados/doacoes_2018.csv',\
                           sep=',',\
                           encoding = 'utf_8',\
                           converters={'SQ_CANDIDATO': lambda x: str(x),
                                       'NR_CPF_CANDIDATO': lambda x: str(x), 
                                       'NR_CPF_CNPJ_DOADOR': lambda x: str(x)},\
                           decimal=',')

def limpeza(linha):
    documento = str(linha["NR_CPF_CNPJ_DOADOR"])
    if len(documento) > 11:
        return documento[:8]
    else:
        return "documento_cpf"

doacoes_2018['cnpj_raiz'] = doacoes_2018.apply(limpeza, axis=1)

In [74]:
doacoes_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321140 entries, 0 to 321139
Data columns (total 21 columns):
SG_UF                          321140 non-null object
DS_CARGO                       321140 non-null object
SQ_CANDIDATO                   321140 non-null object
NR_CANDIDATO                   321140 non-null int64
NM_CANDIDATO                   321140 non-null object
NR_CPF_CANDIDATO               321140 non-null object
NR_CPF_VICE_CANDIDATO          321140 non-null int64
SG_PARTIDO                     321140 non-null object
CD_CNAE_DOADOR                 321140 non-null int64
DS_CNAE_DOADOR                 321140 non-null object
NR_CPF_CNPJ_DOADOR             321140 non-null object
NM_DOADOR                      321140 non-null object
NM_DOADOR_RFB                  321140 non-null object
DS_ESFERA_PARTIDARIA_DOADOR    321140 non-null object
SQ_CANDIDATO_DOADOR            321140 non-null int64
CD_CARGO_CANDIDATO_DOADOR      321140 non-null int64
DS_CARGO_CANDIDATO_DOADOR     

In [75]:
doacoes_2018_pf = doacoes_2018[(doacoes_2018['cnpj_raiz'] == 'documento_cpf')].copy()
doacoes_2018_pf['cpf_mascarado']='***'+doacoes_2018_pf['NR_CPF_CNPJ_DOADOR'].str[3:9]+'**'
doacoes_2018_pf["NM_DOADOR_RFB"] = doacoes_2018_pf["NM_DOADOR_RFB"].apply(f)

Doações diretas de CPF

In [76]:
farma_doadores_diretos_cpf_2018 = pd.merge(doacoes_2018_pf, farma_socios.drop_duplicates('nome_socio'), left_on=['NM_DOADOR_RFB', 'cpf_mascarado'], right_on=['nome_socio', 'cnpj_cpf_do_socio'])

In [77]:
print('O dataframe contém ' + str(farma_doadores_diretos_cpf_2018.shape[0]) + ' linhas')

O dataframe contém 741 linhas


In [78]:
farma_doadores_diretos_cpf_2018 = pd.merge(farma_doadores_diretos_cpf_2018, \
                                       eleitos.drop_duplicates('NR_CPF_CANDIDATO'), \
                                       left_on='NR_CPF_CANDIDATO', \
                                       right_on='NR_CPF_CANDIDATO',\
                                       how='left')

In [79]:
print('O dataframe contém ' + str(farma_doadores_diretos_cpf_2018.shape[0]) + ' linhas')

O dataframe contém 741 linhas


In [80]:
farma_doadores_diretos_cpf_2018.to_excel('resultados/compara/doacoes_diretas_socios_empresas_2018.xlsx',sheet_name='Sheet1')

Maiores doadores

In [81]:
farma_doadores_diretos_cpf_2018.groupby("NM_DOADOR_RFB").VR_RECEITA.sum().reset_index().sort_values("VR_RECEITA", ascending=False).head(10)

,NM_DOADOR_RFB,VR_RECEITA
111,FERNANDO DE CASTRO MARQUES,"7,118,628.89"
280,OGARI DE CASTRO PACHECO,"1,558,550.00"
79,DIANA MARIA NASCIMENTO DA SILVA,"515,000.00"
152,IRIS SCUSSEL STEVANATTO,"370,000.00"
12,ALEXANDRE VICENTE DE PAULA ALMEIDA,"354,316.74"
330,ROGERIO CHOR,"220,000.00"
30,ANGELO MARIO CORONEL DE AZEVEDO MARTINS,"181,000.00"
207,LEANDRA MACHADO SANTOS,"165,000.00"
397,WOLMER DE AZEVEDO ARAUJO,"148,800.00"
109,FAISSAL JORGE CALIL FILHO,"128,158.06"


Candidatos que mais receberam

In [82]:
farma_doadores_diretos_cpf_2018.groupby("NM_CANDIDATO_x").VR_RECEITA.sum().reset_index().sort_values("VR_RECEITA", ascending=False).head(10)

,NM_CANDIDATO_x,VR_RECEITA
119,FERNANDO DE CASTRO MARQUES,"2,705,853.30"
61,CARLOS EDUARDO TORRES GOMES,"2,103,550.00"
329,ROGÉRIO SCHUMANN ROSSO,"889,400.00"
310,RENATO SANTANA DA SILVA,"786,600.00"
52,AUGUSTO SILVEIRA DE CARVALHO,"675,960.00"
202,JULIO CESAR RIBEIRO,"586,140.00"
13,AGENOR GOMES DE ARAÚJO NETO,"400,000.00"
79,DANIEL DE CASTRO SOUSA,"376,000.00"
23,ALEXANDRE VICENTE DE PAULA ALMEIDA,"354,316.74"
78,CRISTOVAM RICARDO CAVALCANTI BUARQUE,"335,780.80"


Tabela dinâmica doadores candidatos empresa

In [83]:
tabela = pd.pivot_table(farma_doadores_diretos_cpf_2018,index=["NM_DOADOR_RFB","NM_CANDIDATO_x", "razao_social"], values=["VR_RECEITA"], aggfunc=np.sum)
result = tabela.sort_values('VR_RECEITA', ascending=False).head(10)
result

VR_RECEITA
NM_DOADOR_RFB                      NM_CANDIDATO_x                       razao_social                                              
FERNANDO DE CASTRO MARQUES         FERNANDO DE CASTRO MARQUES           CLARIS PRODUTOS FARMACEUTICOS DO BRASIL LTDA. 2,705,853.30
OGARI DE CASTRO PACHECO            CARLOS EDUARDO TORRES GOMES          CONSORCIO CNG                                 1,558,550.00
FERNANDO DE CASTRO MARQUES         ROGÉRIO SCHUMANN ROSSO               CLARIS PRODUTOS FARMACEUTICOS DO BRASIL LTDA.   889,400.00
                                   RENATO SANTANA DA SILVA              CLARIS PRODUTOS FARMACEUTICOS DO BRASIL LTDA.   786,600.00
                                   AUGUSTO SILVEIRA DE CARVALHO         CLARIS PRODUTOS FARMACEUTICOS DO BRASIL LTDA.   675,960.00
                                   JULIO CESAR RIBEIRO                  CLARIS PRODUTOS FARMACEUTICOS DO BRASIL LTDA.   566,140.00
                                   DANIEL DE CASTRO SOUSA               CLARIS PRODUTOS FARMACEUTICOS DO BRASIL LTDA.   376,000.00
ALEXANDRE VICENTE DE PAULA ALMEIDA ALEXANDRE VICENTE DE PAULA ALMEIDA   A H P SANTOS DE MACEDO & CIA LTDA               354,316.74
FERNANDO DE CASTRO MARQUES         CRISTOVAM RICARDO CAVALCANTI BUARQUE CLARIS PRODUTOS FARMACEUTICOS DO BRASIL LTDA.   335,780.80
DIANA MARIA NASCIMENTO DA SILVA    AGENOR GOMES DE ARAÚJO NETO          NUVEX COMERCIO DE PRODUTOS MEDICOS LTDA         320,000.00

In [84]:
selecao1 = farma_doadores_diretos_cpf_2018.drop_duplicates(['nome_socio'], keep='first')
selecao2 = selecao1[['nome_socio', 'cnpj_cpf_do_socio']].copy()
selecao3 = pd.merge(farma_socios, selecao2, left_on=['nome_socio', 'cnpj_cpf_do_socio'], right_on=['nome_socio', 'cnpj_cpf_do_socio'])
selecao4 = selecao3[['cnpj', 'razao_social', 'nome_socio', 'cnpj_cpf_do_socio']].copy()
selecao4['nome_socio'].value_counts()

EUGENIO DE ZAGOTTIS                               2017
RODRIGO DE ALMEIDA PIZZINATTO                      536
JULIO RICARDO ANDRIGHETTO MOTTIN                   450
ROBERTO LUIZ WEBER                                 450
PAULO REMY GILLET NETO                             255
MODESTO CARVALHO DE ARAUJO NETO                    221
PAULO CESAR MATTAR                                 102
OSMAR FERNANDES DE SOUZA                            68
SEVERINO RODRIGUES DA SILVA                         45
RUBENS BENEDITO AUGUSTO                             41
MIRIAN ROSE AUGUSTO                                 36
JORGE PEREIRA DE SOUZA                              34
ROBERTOMAURO PEREIRA DE SOUZA                       34
ALEXANDRE PEREIRA DE SOUZA NETTO                    34
FERNANDO DE CASTRO MARQUES                          29
VIVIANY GUIMARAES DA FONSECA ALEXANDRE              27
SANDRA REGINA WENDLAND                              21
RUBIN WENDLAND                                      21
ALVARO SIL

In [85]:
selecao4.to_excel('resultados/compara/empresas_e_socios_das_empresas_de_2018.xlsx',sheet_name='Sheet1')